# CUMULATIVE GDD IN TOMATO FARM

The cumulative Growing Degree Days (GDD) of each block in Toguchi Farm


In [1]:
import pandas as pd
import numpy as np
import pymysql
pymysql.install_as_MySQLdb()
import sys
import datetime as dt
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models.widgets import Tabs, Panel
import pandas, datetime
from time import gmtime,strftime
#from calendar import timegm
from bokeh.layouts import row, column, gridplot
from bokeh.models import ColumnDataSource,HoverTool,Range1d

In [2]:
# Connection to MySQL Database to get the data
conn = pymysql.connect(host = 'myHost', port= MyPort, user ='myRoot', passwd = 'MyPwd', db = 'MyDB')

cursor1 = conn.cursor()
cursor2 = conn.cursor()

cursor1.execute('SELECT sdate,ROUND((max_temp+min_temp)/2,1), sensor FROM daily_temp ORDER BY sdate ASC')
cursor2.execute('SELECT DISTINCT block,vid,planting, harvesting FROM toguchi_plant WHERE  planting IN (SELECT Planting FROM toguchi_plant) ORDER BY planting DESC')
result1 = cursor1.fetchall()
result2 = cursor2.fetchall()

df1 = pd.DataFrame( [[ij for ij in i] for i in result1] )
df2 = pd.DataFrame( [[ij for ij in i] for i in result2] )

df1.rename(columns={0: 'Date', 1:'Mean_T', 2:'sensor'}, inplace=True);
df2.rename(columns={0: 'block', 1: 'vid', 2: 'startPlant', 3: 'startHarv'}, inplace=True);
if cursor1 and conn:
    cursor1.close()
if cursor2 and conn:
    cursor2.close()
    conn.close()

In [3]:
df1.head()

,Date,Mean_T,sensor
0,2017-04-28,23.2,02:00:00:28:8c:e6
1,2017-04-28,23.2,03:00:00:02:c5:f8
2,2017-04-28,23.2,03:00:00:05:05:30
3,2017-04-29,21.9,02:00:00:28:8c:e6
4,2017-04-29,21.9,03:00:00:02:c5:f8


In [4]:
df2.head()

,block,vid,startPlant,startHarv
0,C,v1,2017-07-18,2017-09-18
1,D,v1,2017-05-26,2017-07-25
2,A,v2,2017-04-27,2017-06-19
3,E,v1,2017-03-17,None
4,F,v1,2017-02-21,2017-05-08


### SENSORS USED IN TOGUCHI FARM

    Toguchi Farm 2	OutDoor	NURS_N          02:00:00:27:38:c2 
    Toguchi Farm 2	Indoor	D-S             03:00:00:02:c5:f8
    Toguchi Farm 2	Base Station	D-N     70:ee:50:27:49:2e
    Toguchi Farm 3	OutDoor	E-S             02:00:00:27:39:82
    Toguchi Farm 3	Indoor	F-N             03:00:00:04:f2:00
    Toguchi Farm 3	Indoor	F-S             03:00:00:04:1a:d0
    Toguchi Farm 3	Base Station	E-N     70:ee:50:27:2e:da
    Toguchi Farm 4	OutDoor	AB-S            02:00:00:28:8c:e6
    Toguchi Farm 4	Indoor	CB-S            03:00:00:05:05:30
    Toguchi Farm 4	Base Station	CB-N    70:ee:50:28:e8:40
    Toguchi Farm 4	Indoor	AB-N	        i053710

In [5]:

df2_A = df2.query('block == "A" & vid == "v2"')
df1_A = df1.query('sensor == "02:00:00:28:8c:e6"')

df2_B = df2.query('block == "B" & vid == "v1"')
df1_B = df1.query('sensor == "70:ee:50:27:2e:da"')

df2_C = df2.query('block == "C" & vid == "v1"')
df1_C= df1.query('sensor == "03:00:00:05:05:30"')

df2_D = df2.query('block == "D" & vid == "v1"')
df1_D = df1.query('sensor == "03:00:00:02:c5:f8"')

df2_E = df2.query('block == "E" & vid == "v1"')
df1_E = df1.query('sensor == "02:00:00:27:39:82"')

df2_F = df2.query('block == "F" & vid == "v1"')
df1_F = df1.query('sensor == "03:00:00:04:f2:00"')


In [6]:
#Drop
clean_df1_A = df1_A.drop('sensor',1)
clean_df1_B = df1_B.drop('sensor',1)
clean_df1_C = df1_C.drop('sensor',1)
clean_df1_D = df1_D.drop('sensor',1)
clean_df1_E = df1_E.drop('sensor',1)
clean_df1_F = df1_F.drop('sensor',1)

In [7]:
clean_df2_A = df2_A.drop(['block','vid'], 1)
clean_df2_B = df2_B.drop(['block','vid'], 1)
clean_df2_C = df2_C.drop(['block','vid'], 1)
clean_df2_D = df2_D.drop(['block','vid'], 1)
clean_df2_E = df2_E.drop(['block','vid'], 1)
clean_df2_F = df2_F.drop(['block','vid'], 1)

In [8]:
clean_df2_A
#clean_df1_A

,startPlant,startHarv
2,2017-04-27,2017-06-19
14,None,None


In [9]:
def df1_df2(df11,df22):
    
    startHarv_A= df22['startPlant'].iloc[0]
    #endHarv    = df22['startHarv'].iloc[0]

    endHarv = dt.date.today()
    
    full_range = pd.date_range(startHarv_A, endHarv)
    
    df11.set_index(['Date'], inplace=True)
    df_complete = df11.sort_index().reindex(full_range).interpolate(method='linear')
    return df_complete

In [10]:
clean_df2_A
#clean_df1_A

,startPlant,startHarv
2,2017-04-27,2017-06-19
14,None,None


In [11]:
df_intepl = df1_df2(clean_df1_A,clean_df2_A)
df_intep2 = df1_df2(clean_df1_B,clean_df2_B)
df_intep3 = df1_df2(clean_df1_C,clean_df2_C)
df_intep4 = df1_df2(clean_df1_D,clean_df2_D)
df_intep5 = df1_df2(clean_df1_E,clean_df2_E)
df_intep6 = df1_df2(clean_df1_F,clean_df2_F)

In [12]:
# Function to Calculte GDD (Growing Degree Degree)
def HDH (Tcurr,Tref=10.0):
    if Tref <= Tcurr:
        return ((Tcurr-Tref))
    else:
        return (0)

In [13]:
df_intepl['GDD'] = df_intepl.apply(lambda row: HDH(row['Mean_T']),axis=1)
df_intep2['GDD'] = df_intep2.apply(lambda row: HDH(row['Mean_T']),axis=1)
df_intep3['GDD'] = df_intep3.apply(lambda row: HDH(row['Mean_T']),axis=1)
df_intep4['GDD'] = df_intep4.apply(lambda row: HDH(row['Mean_T']),axis=1)
df_intep5['GDD'] = df_intep5.apply(lambda row: HDH(row['Mean_T']),axis=1)
#df_intep6['GDD'] = df_intep6.apply(lambda row: HDH(row['Mean_T']),axis=1)


In [14]:
#Function to calculate Cumulative GDD
def full_df(df22):
    df22 = df22.reset_index(drop=False)
    df22['Cum_GDD'] = df22.GDD.cumsum()
    df = df22
    return df

In [15]:
dfA = full_df(df_intepl)
dfB = full_df(df_intep2)
dfC = full_df(df_intep3)
dfD = full_df(df_intep4)
dfE = full_df(df_intep5)
#dfF = full_df(df_intep6)

In [16]:
#FUNCTION To VISUALIZE the result Using Bokeh Library
def plotTab(df,k):
    Date = df.index
    GDD = df.GDD
    Cum_GDD = df.Cum_GDD
    
    source = ColumnDataSource(
        data=dict(
            GDD = GDD,
            Cum_GDD = Cum_GDD,
            Date = pd.to_datetime(df['index'], format='%Y-%m-%d'),
        )
    )
    
    source.add(df['index'].apply(lambda d: d.strftime('%Y-%m-%d')), 'date_formatted')
    
    #Hover with specfied tooltips
    hover = HoverTool(
        tooltips="""
             <div style="background: #FFFFFF;">
                <span style="font-size: 20px; color: blue;"> cumulative GDD, @Cum_GDD{int}</span><br />
                <span style="font-size: 15px; color: green;"> DATE, @date_formatted</span><br />

            </div>
        """)
    
    # Create figures
    p = figure(x_axis_label='DATE', x_axis_type='datetime',
                y_axis_label='CUMMULATIVE GDD',
                background_fill_color='#D1D1D8',
                tools=[hover, 'xpan','reset','previewsave'],
                #plot_width=1000,
                logo=None)
    p.line(x='Date', y='Cum_GDD',source=source,line_width=1,line_color="blue")
    p.circle('Date','Cum_GDD',source=source)
    p.xaxis.axis_label_text_font_size = "20pt"
    p.yaxis.axis_label_text_font_size = "20pt"

    #Create
    tab = Panel(child=p, title= k)
    return tab

In [18]:
tab1 = plotTab(dfA,'BLOCK A')
tab2 = plotTab(dfB,'BLOCK B')
tab3 = plotTab(dfC,'BLOCK C')
tab4 = plotTab(dfD,'BLOCK D')
tab5 = plotTab(dfE,'BLOCK E')
#tab6 = plotTab(dfF,'BLOCK F')
def layoutResult():
    layout = Tabs(tabs=[tab1, tab2, tab3, tab4, tab5]) #tab6])
    output_file('CUMULATIVE_GDD_TOGU.html')

    show (layout)
    return layout 
output_notebook()
layoutResult()

Loading BokehJS ...

Tabs(id='0f549eb4-74b5-44e5-90e3-897da7d7260f', ...)